<a href="https://colab.research.google.com/github/Lupama2/Maestria-Bunkin/blob/main/Dependencia_laser_Rmax/Simulacion_particulas_equlibrio/simulacion_de_particulas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulación de la evolución de la avalancha de electrones

El objetivo es simular la evolución de la avalancha de electrones descripta en el paper de Bunkin desde la presencia de un único electrón. Para ello se cuenta con las ecuaciones de movimiento dadas por la ley de Newton y la ley de Lorentz, adimensionalizadas (ver cuadernillo). Debido a la simetría del problema dado que el campo E no tiene dependencia espacial, el sistema es bidimensional. Además, como condición inicial se tiene a un único electrón con velocidad correspondiente a $T_e = 2$ eV y ubicación en un círculo de radio R0 aleatoria.

La condición para la reproducción del nro de electrones (crecimiento de la avalancha) está dado por los choques con las paredes. En estos existe una probabilidad w


Si lo hago en C++ creo que sería mucho más fácil crear un objeto partícula

Los datos con sufijo problema_energia hacen referencia a cuando la energía que tiene una partícula nueva es mayor a la que debería tener debido a que no se consideró la energía potencial que adquiere.







PROBLEMA CUDA:

Fabián me recomendó trabajar con un problema sencillo para CUDA. Elijo trabajar con la avalancha de electrones PERO sin la posibilidad de que haya reproducción de electrones y sin campo eléctrico aplicado

Sean N electrones en una cavidad circular (2D) inicialmente a temperatura T = 5000 K. Asumiendo que las colisiones entre ellos están mediadas por el potencial de coulomb y que las colisiones con la pared son elásticas, ¿cómo es la distribución espacial de las partículas en el equilibrio?

Comienzo con N partículas distribuídas aleatoriamente en la cavidad, cuyas velocidades corresponden a una distribución de maxwell boltzmann de temperatura T = 5000 K

Calculo las trayectorias de las partículas usando Ley de Newton ++ Ley de Lorentz

Condición de pared blanda para la pared

Evoluciono el sistema y calculo para cada tiempo la temperatura, varía? será un buen parámetro para determinar la condición de equilibrio? ¿Cómo evoluciona el perfil de densidad (que solo depende del radio)?

## Importo dependencias

In [11]:
#Importo librerías
import numpy as np

## Defino ctes

In [12]:
#Constantes matemáticas
pi = np.pi

#Ctes físicas
m = 9.11e-31*1e3 #[g]
e = 1.602e-19*(1/3.336641e-10) #[Fr]
c = 299792458*1e2 #[cm/s]
K = 1.380649e-23*(1/1e-7) #constante de Boltzmann [ergio/K], obtenida de NIST


## C.I. (Condiciones Iniciales)

A partir de ahora todas las variables definidas son adimensionales. Todas las variables dimensionales se aclararán con el sufico _dim

El campo está en la dirección y

In [13]:
#Radio del círculo y velocidad inicial de la partícula
R0_dim = 1e-6 #[cm]
T0_dim = 300 #[K]

In [14]:
#Calculo las ctes adimensionales
v0_dim = np.sqrt(3*K*T0_dim/m)
# rho1 = v0_dim/R0_dim
# rho2 = e**2/(R0_dim**2*m*v0_dim)
alpha = e**2/m/R0_dim/(v0_dim**2)
print(f"Constante adimensional, alpha = {alpha}")

#Radio del círculo y velocidad inicial de la partícula adimensionales
R0 = 1
v0 = 1

print("Radio y velocidad iniciales adimensionales: ", R0, ",\t", v0)

Constante adimensional, alpha = 1.8551552825550501
Radio y velocidad iniciales adimensionales:  1 ,	 1


## Sistema de ecuaciones diferenciales

In [15]:
def f(y, N):
    '''
    Parameters
    ----------
    y_vec (ndarray de dimensión 4N): vector de variables formado por [rx_vec, ry_vec, vx_vec, vy_vec] (en ese orden). Estos son:
        rx_vec (ndarray de dimensión N): vector de posiciones en x de las partículas
        ry_vec (ndarray de dimensión N): vector de posiciones en y de las partículas
        vx_vec (ndarray de dimensión N): vector de velocidades en x de las partículas
        vy_vec (ndarray de dimensión N): vector de velocidades en y de las partículas
    q_vec (ndarray de dimensión N): vector de cargas adimensionales de las partículas
    t (float): tiempo

    Nota:
    N (int): nro de partículas
    '''
    rx_vec = y[:N]
    ry_vec = y[N:2*N]
    vx_vec = y[2*N:3*N]
    vy_vec = y[3*N:4*N]


    #Calculo drdt
    drx_vec = vx_vec
    dry_vec = vy_vec

    #Calculo dvdt
    Ax_matriz = np.tile(rx_vec, (N,1))
    Ay_matriz = np.tile(ry_vec, (N,1))

    Bx_matriz = Ax_matriz.T - Ax_matriz
    By_matriz = Ay_matriz.T - Ay_matriz

    modulo = np.sqrt(Bx_matriz**2 + By_matriz**2)
    modulo = np.where(modulo == 0, 1, modulo) #Para evitar división por cero

    Cx_matriz = Bx_matriz/modulo**3
    Cy_matriz = By_matriz/modulo**3

    dvx_vec = alpha*np.sum(Cx_matriz, axis=1)
    dvy_vec = alpha*np.sum(Cy_matriz, axis=1)
   
    dydt = np.concatenate((drx_vec, dry_vec, dvx_vec, dvy_vec))

    return dydt



In [16]:
from scipy.stats import maxwell

def f_maxwell(N):
    factor_Chi_to_v = 1/np.sqrt(m/(K*T0_dim))
    return maxwell.rvs(N)*factor_Chi_to_v/v0_dim


def condiciones_iniciales(N):
    '''
    Devuelve y0, q0 tal que existen n_ini partículas iniciales de N partículas posibles consideradas.
    Las posiciones son aleatorias. La dirección de la velocidad también lo es, pero el módulo de la velocidad inicial en todas las partículas es el mismo e igual a v0
    '''
    y0 = np.empty(4*N)

    #Posiciones aleatorias
    r0_vec = R0*np.random.rand(N)
    tita0_vec = 2*pi*np.random.rand(N)
    y0[:N] = r0_vec*np.cos(tita0_vec) # = rx0_vec[i]
    y0[N:2*N] = r0_vec*np.sin(tita0_vec) # = ry0_vec[i] 

    #Velocidades de acuerdo a una distribución de Maxwell-Boltzmann 
    v0_vec = f_maxwell(N)
    tita0_vec = 2*pi*np.random.rand(N)
    y0[2*N:3*N] = v0_vec*np.cos(tita0_vec) # = vx0_vec[i]
    y0[3*N:4*N] = v0_vec*np.sin(tita0_vec) # = vy0_vec[i]

    return y0


In [17]:
def distancia_al_origen(r_vec, N):
    '''
    Calcula la distancia al origen de las partícula
    r = [rx, ry]

    REVISAR
    '''

    #Opero de forma similar a como lo hice dentro de f
    rx = r_vec[:N]
    ry = r_vec[N:]

    d_vec = np.sqrt(rx**2 + ry**2)

    return d_vec

## Solución numérica

In [18]:
def rebote_duro(rx, ry, vx, vy):

    '''
    Calcula las posiciones y velocidades de la partícula luego del rebote con la pared
    rx_1new, ry_1new, vx_1new, vy_1new = rebote(rx, ry, vx_1, vy_1)

    Se considera la pared como dura
    '''


    #Calculo la nueva posición
    factor = -1 + 2*R0/np.sqrt(rx**2 + ry**2)

    rx_new = rx*factor
    ry_new = ry*factor

    #Calculo la nueva velocidad
    
    #Calculo el ángulo del vector [rx,ry]
    tita = np.arctan2(ry, rx)
    vx_new = -vx*np.cos(2*tita) - vy*np.sin(2*tita)
    vy_new = -vx*np.sin(2*tita) + vy*np.cos(2*tita)

    return rx_new, ry_new, vx_new, vy_new

def rebote_blando(rx, ry, vx, vy):

    '''
    Calcula las posiciones y velocidades de la partícula luego del rebote con la pared
    rx_1new, ry_1new, vx_1new, vy_1new = rebote(rx, ry, vx_1, vy_1)

    Se considera la pared como blanda (ver FISCOM)
    '''

    #Calculo la nueva posición
    rx_new = rx
    ry_new = ry

    #Calculo la nueva velocidad
    
    #Calculo el ángulo del vector [rx,ry]
    tita = np.arctan2(ry, rx)
    vx_new = -vx*np.cos(2*tita) - vy*np.sin(2*tita)
    vy_new = -vx*np.sin(2*tita) + vy*np.cos(2*tita)

    return rx_new, ry_new, vx_new, vy_new


def metodo_Euler(yold, t, dt, N):
    '''
    Método de Euler
    '''
    dydt = np.zeros(4*N)
    ynew = yold + dt*f(yold, N)
    return ynew

def metodo_Verlet(yold, t, dt, N):
    '''
    Método de Verlet definido en el apunte de FISCOM
    '''
    
    #Calculo el vector de fuerzas
    dydt = f(yold, N) 
    F_vec = dydt[2*N:]

    #Asigno el vector de posiciones
    r_vec = yold[:2*N]
    #Calculo la posición en el siguiente paso de tiempo
    r_vec_new = r_vec + yold[2*N:4*N]*dt + 1/2*dt**2*F_vec

    #Calculo la fuerza en el siguiente paso de tiempo
    ynew_partial = np.concatenate((r_vec_new, yold[2*N:])) #corrijo solo r_vec

    dydt_new = f(ynew_partial, N)
    F_vec_new = dydt_new[2*N:]

    #Calculo la velocidad en el siguiente paso de tiempo
    v_vec_new = yold[2*N:] + 1/2*dt*(F_vec + F_vec_new)

    return np.concatenate((r_vec_new, v_vec_new))



def avanzo_dt(y, t, dt, N, metodo):
    '''
    Avanza la solución en un paso de tiempo
    '''
    #Avanzo un paso de tiempo
    ynew = metodo(y, t, dt, N)

    #Verifico si se cumple la condición de choque
    #Calculo la distancia de cada partícula al origen
    d_vec = distancia_al_origen(ynew[0:2*N], N)

    #Determino todos los índices en los que una partícula superó la distancia R0
    indices = np.where(d_vec>R0)[0]
    #Opero sobre las partículas que chocaron
    if len(indices) > 0:
        for indice in indices:
            #Calculo las variables correspondientes
            rx = ynew[indice]
            ry = ynew[indice+N]
            vx = ynew[indice+2*N]
            vy = ynew[indice+3*N]

            #Rebota
            rx_new, ry_new, vx_new, vy_new = rebote_blando(rx, ry, vx, vy)
            #Añado los nuevos datos en ynew de forma ordenada
            ynew[indice] = rx_new
            ynew[indice+N] = ry_new
            ynew[indice+2*N] = vx_new
            ynew[indice+3*N] = vy_new

    return ynew



In [19]:
guardo_cada = 10000 #100

In [20]:
#Hago una simulación
N = 100

y0 = condiciones_iniciales(N)
y = y0.copy()
t = 0
dt = 1e-8 #En 1e-1 ya genera problemas. Las partículas salen de la circunferencia
#A mayor N, menor dt
n_pasos = 10000*10 #20000*5*10

# datos = np.empty([n_pasos, N_esperado*4])
pos_x = np.empty([n_pasos//guardo_cada, N])
pos_y = np.empty([n_pasos//guardo_cada, N])
vel_x = np.empty([n_pasos//guardo_cada, N])
vel_y = np.empty([n_pasos//guardo_cada, N])

#Inicializo


for i in range(n_pasos):
    if i%(n_pasos//500) == 0:
        print(f"t = {round(t,2)}\tEvolución al {round(i/n_pasos*100,2)}%")

    t += dt
    if i%guardo_cada == 0:
        pos_x[i//guardo_cada] = y[0:N]
        pos_y[i//guardo_cada] = y[N:2*N]
        vel_x[i//guardo_cada] = y[2*N:3*N]
        vel_y[i//guardo_cada] = y[3*N:4*N]

    y = avanzo_dt(y, t, dt, N, metodo_Verlet)


t = 0	Evolución al 0.0%
t = 0.0	Evolución al 0.2%
t = 0.0	Evolución al 0.4%
t = 0.0	Evolución al 0.6%
t = 0.0	Evolución al 0.8%
t = 0.0	Evolución al 1.0%
t = 0.0	Evolución al 1.2%
t = 0.0	Evolución al 1.4%
t = 0.0	Evolución al 1.6%
t = 0.0	Evolución al 1.8%
t = 0.0	Evolución al 2.0%
t = 0.0	Evolución al 2.2%
t = 0.0	Evolución al 2.4%
t = 0.0	Evolución al 2.6%
t = 0.0	Evolución al 2.8%
t = 0.0	Evolución al 3.0%
t = 0.0	Evolución al 3.2%
t = 0.0	Evolución al 3.4%
t = 0.0	Evolución al 3.6%
t = 0.0	Evolución al 3.8%
t = 0.0	Evolución al 4.0%
t = 0.0	Evolución al 4.2%
t = 0.0	Evolución al 4.4%
t = 0.0	Evolución al 4.6%
t = 0.0	Evolución al 4.8%
t = 0.0	Evolución al 5.0%
t = 0.0	Evolución al 5.2%
t = 0.0	Evolución al 5.4%
t = 0.0	Evolución al 5.6%
t = 0.0	Evolución al 5.8%
t = 0.0	Evolución al 6.0%
t = 0.0	Evolución al 6.2%
t = 0.0	Evolución al 6.4%
t = 0.0	Evolución al 6.6%
t = 0.0	Evolución al 6.8%
t = 0.0	Evolución al 7.0%
t = 0.0	Evolución al 7.2%
t = 0.0	Evolución al 7.4%
t = 0.0	Evoluc

In [21]:
# Sigo corriendo el programa


# n2_pasos = 2*n_pasos #Correspondientes al segundo ciclo de evolución

# pos_x = np.concatenate((pos_x, np.empty([n2_pasos//guardo_cada, N_esperado])))
# pos_y = np.concatenate((pos_y, np.empty([n2_pasos//guardo_cada, N_esperado])))
# vel_x = np.concatenate((vel_x, np.empty([n2_pasos//guardo_cada, N_esperado])))
# vel_y = np.concatenate((vel_y, np.empty([n2_pasos//guardo_cada, N_esperado])))
# q_tot = np.concatenate((q_tot, np.zeros([n2_pasos//guardo_cada, N_esperado])))


# for i in range(n_pasos, n_pasos + n2_pasos):
#     try:
#         y = avanzo_dt(y, q_vec, t, dt, metodo)
#     except ValueError:
#         print(f"Último índice: {i}")
#         break

#     if i%guardo_cada == 0:
#         print(f"t = {round(t,2)}\tEvolución al {round(i/(n_pasos+n2_pasos)*100,2)}%\tN = {np.sum(q_vec)}")


#     t += dt
#     if i%guardo_cada == 0:
#         pos_x[i//guardo_cada] = np.concatenate((y[0:len(y)//4], np.zeros( N_esperado - len(y)//4)))
#         pos_y[i//guardo_cada] = np.concatenate((y[len(y)//4:2*len(y)//4], np.zeros( N_esperado - len(y)//4)))
#         vel_x[i//guardo_cada] = np.concatenate((y[2*len(y)//4:3*len(y)//4], np.zeros( N_esperado - len(y)//4)))
#         vel_y[i//guardo_cada] = np.concatenate((y[3*len(y)//4:4*len(y)//4], np.zeros( N_esperado - len(y)//4)))
#         q_tot[i//guardo_cada] = q_vec

# n_pasos = n2_pasos + n_pasos

# print("N_tot = ", np.sum(q_vec))


In [22]:
#Exporto las trayectorias
np.array([pos_x, pos_y, vel_x, vel_y]).dump('resultados/trayectorias.npy')
#Exporto las condiciones iniciales
np.array([R0, v0, R0_dim, v0_dim]).dump("resultados/condiciones_iniciales.npy")
#Exporto los tiempos
np.array([t, dt, n_pasos, guardo_cada]).dump("resultados/tiempos.npy")